# 生成部分代码功能
## 将MNIST数据集转换为TFRecord格式 步骤如下:
** 将数据填入example协议内存块 **  
** 将协议内存块序列化为一个字符串 **  
** 通过python_io.TFRecordWriter类写入到TFRecord文件 **

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist= input_data.read_data_sets('/input/MNIST_data',dtype=tf.uint8,one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /input/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /input/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /input/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting /input/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


#### 定义生成整数型和字符串型属性的方法，这是将数据填入到Example协议内存块 (protocol buffer)的第一步，以后会调用到这个方法

In [4]:
def Int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def Bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

#### 读取mnist数据。

In [8]:
images = mnist.train.images
labels = mnist.train.labels
pixels = images.shape[1]
num_examples = mnist.train.num_examples

#### 输出TFRecord文件的地址(相对于系统根目录)。

In [14]:
filename='./output/MNIST_tfrecords'

#### 创建一个python_io.TFRecordWriter()类的实例

In [15]:
writer=tf.python_io.TFRecordWriter(filename)

#### for循环执行了将数据填入到Example协议内存块的主要操作

In [23]:
for i in range(num_examples):
    # 将图像矩阵转化成一个字符串
    image_to_string = images[i].tostring()
    feature = {
        'pixels':Int64_feature(pixels),
        'label':Int64_feature(np.argmax(labels[i])),
        'image_raw':Bytes_feature(image_to_string)
    }
    
    features = tf.train.Features(feature=feature)
    # 定义一个Example，将相关信息写入到这个数据结构
    
    example=tf.train.Example(features=features)
    
    # 将一个Example写入到TFRecord文件
    # 原型writer(self, record)
    writer.write(example.SerializeToString())
# 在写完文件后最好的习惯是调用close()函数关闭
writer.close()

# 解析部分代码功能
## 从TFRecord文件读取数据，可以使用  *  TFRecord 类* 中的  * parse_single_example() *函数作为解析器，该函数会将Example协议内存块解析为张量

#### 创建一个TFRecordReader类的实例

In [26]:
reader=tf.TFRecordReader()


#### 创建一个队列对输入文件列表进行维护
#### 函数原型string_input_producer(string_tensor,num_epochs,shuffle,seed, capacity,shared_name,name,cancel_op)

In [27]:
filename_queue=tf.train.string_input_producer(['./output'])

#### 使用TFRecordReader.read()函数从文件中读取一个样例，原型reader(self,queue,name)
#### 也可使用read_up_to()函数一次性读取多个样例，
#### 原型read_up_to(self,queue,num_records,name)

In [28]:
_,serialized_example=reader.read(filename_queue)

#### 使用parse_single_example()函数解析读取的样例。
#### 原型parse_single_example(serialized,features,name,example_names)

In [30]:
features=tf.parse_single_example(
                serialized_example,
                features={
                        #可以使用FixedLenFeature类对属性进行解析，
                        'image_raw':tf.FixedLenFeature([],tf.string),
                        'pixels':tf.FixedLenFeature([],tf.int64),
                        'label':tf.FixedLenFeature([],tf.int64)
        
                }
    
        )

#### decode_raw()函数用于将字符串解析成图像对应的像素数组
#### 函数原型decode_raw(bytes,out_type,little_endian,name)

In [31]:
images=tf.decode_raw(features['image_raw'],tf.uint8)

#### 使用cast()函数进行类型转换

In [35]:
labels = tf.cast(features['label'],tf.int32)
pixels = tf.cast(features['pixels'],tf.int32)

In [40]:
with tf.Session() as sess:
    #启动多线程处理输入数据
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coordinator)
    for i in range(10):
        image,label,pixel=sess.run([images,labels,pixels])
        print(label)

7
3
4
6
1
8
1
0
9
8
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer, input_producer/Const, ^input_producer/Assert/Assert)]]


In [ ]:
tf.